# CS480: Database Systems, Group Project
### Green Taxi Datasets

In [ ]:
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import sqlite3
from hopcroftkarp import HopcroftKarp
import time
from itertools import chain
from functools import reduce

#### 1. First, we will import the data in the system

In [ ]:
green = pd.read_csv('green_tripdata_2015_6months_cleaned.csv')

#### 2. Now, we will get the summary of the datasets and then we will clean up the dataset

In [ ]:
green.describe()
# print(green.dtypes)

In [ ]:
# Getting the values which needs to be cleaned up before procedding further
print("\u0332".join('Number of Null data values in each columns:'))
print(green.isnull().sum())
print('')
print("\u0332".join('Number of Datasets:'), len(green.index))

In [ ]:
# Since the number of null values in the 'Ehail_fee' column is equal to the number of rows in the dataset
# Therefore, we will drop the whole column due to its irrelevance.

# green = green.drop(columns=['Ehail_fee'])
# print("\u0332".join('Number of Null data values in each columns:'))
# print(green.isnull().sum())

In [ ]:
# Now we will boxplot the 'Trip_type' column
# boxplot = green.boxplot(column=['Trip_type '])

In [ ]:
# In the boxplot, we can see that '2.0' is an outlier, therefore majority of the dataset have value '1'
# So, we will replace the null values with '1'
# green['Trip_type '] = green['Trip_type '].fillna(1)

In [ ]:
# Now there are no null values in the dataset
# Update the columns variable according to the new cleaned up value
# columns = green.columns
# print("\u0332".join('Number of Null data values in each columns:'))
# print(green.isnull().sum())

#### 3. Now, filter out the data and clean it up again and to get the data we need use for our algorithms

In [ ]:
dataset = green[['VendorID', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime', 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude', 'Passenger_count', 'Trip_distance']]

# Here we are storing all the column names in a array named 'columns'
columns = dataset.columns
print("\u0332".join('Column Names:'), columns)

print("\u0332".join('Dataset:'), len(dataset.index))
dataset = dataset[ (dataset.lpep_pickup_datetime != dataset.Lpep_dropoff_datetime) & (dataset.Trip_distance != 0) & (dataset.Passenger_count < 3) & (dataset['Trip_distance'] != dataset['Trip_distance'].max())]

print("\u0332".join('Filtered Dataset:'), len(dataset.index))

In [ ]:
print("\u0332".join('Total Distance Travelled:'), dataset['Trip_distance'].sum())

In [ ]:
# Correcting the date and time format
dataset['lpep_pickup_datetime'] = pd.to_datetime(dataset['lpep_pickup_datetime'])
dataset['Lpep_dropoff_datetime'] = pd.to_datetime(dataset['Lpep_dropoff_datetime'])
dataset.head()

#### 4. Now use SQL Queries to compute the average speed of each trips

In [ ]:
# conn = sqlite3.connect('TestDB1.db')
# c = conn.cursor()

# dataset.to_sql('Green', conn, if_exists='replace', index = False)
 
# c.execute('''  
# SELECT COUNT(*) FROM Green
#           ''')

# print(c.fetchall())

In [ ]:
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lon1, lat2, lon2): 
    
#     start = time.time()
    
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 

    c = (3956 * 2 * asin(sqrt(sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2)))

#     end = time.time()
#     print(end-start)
    
    return c
     
    # Radius of earth in kilometers. Use 3956 for miles 
    # calculate the result 
    # (c * r) 

def time_difference(time1, time2):
    time_diff = (time2 - time1)
    time_seconds = abs(time_diff.total_seconds())
    
    return time_seconds / 60.0

print(columns)

In [ ]:
dataset = dataset.sort_values(by=['lpep_pickup_datetime', 'Lpep_dropoff_datetime'])
# dataset = dataset.reset_index()

In [ ]:
dataset['Average Speed (MPH)'] = dataset.apply(lambda row : row.Trip_distance / (time_difference(row.lpep_pickup_datetime, row.Lpep_dropoff_datetime) / 60.0), axis = 1) 

In [ ]:
dataset = dataset.reset_index(drop=True)

In [ ]:
dataset

In [ ]:
def mergeable_check(trip1, trip2, delay):
    
#     avg_speed = (trip1['Average Speed (MPH)'] + trip2['Average Speed (MPH)'])/2
    
    Do1o2 = distance(trip1['Pickup_latitude'], trip1['Pickup_longitude'], trip2['Pickup_latitude'], trip2['Pickup_longitude'])
    Dd1d2 = distance(trip1['Dropoff_latitude'], trip1['Dropoff_longitude'], trip2['Dropoff_latitude'], trip2['Dropoff_longitude'])

    Do2d1 = distance(trip2['Pickup_latitude'], trip2['Pickup_longitude'], trip1['Dropoff_latitude'], trip1['Dropoff_longitude'])
    Do2d2 = distance(trip2['Pickup_latitude'], trip2['Pickup_longitude'], trip2['Dropoff_latitude'], trip2['Dropoff_longitude'])
    
#     To1o2 = Do1o2/avg_speed
#     Td1d2 = Dd1d2/avg_speed
    
#     To2d1 = Do2d1/avg_speed
#     To2d2 = Do2d2/avg_speed
    
#     print(max([Do2d1, Do2d2]) + Do1o2 + Dd1d2, "|", trip1['Trip_distance'] + trip2['Trip_distance'])
    
#     sequence1 = False
#     sequence2 = False
    
#     sequence1Time = Trip1['lpep_pickup_datetime']
#     sequence2Time = Trip2['lpep_pickup_datetime']
    
    dist = max([Do2d1, Do2d2]) + Do1o2 + Dd1d2
    if (dist < trip1['Trip_distance'] + trip2['Trip_distance']):
#         print('True', dist, trip1['Trip_distance'] + trip2['Trip_distance'])
        return (True, (trip1['Trip_distance'] + trip2['Trip_distance']) - dist)
    else:
#         print('False', dist, trip1['Trip_distance'] + trip2['Trip_distance'])
        return (False, 0)
    
#     if (sequence1 or sequence2): return True

In [ ]:
trips_processed = 0
mergeable_trips = 0

def shared_trips_eval(dataset, delay, length):    
    
    global trips_processed 
    global mergeable_trips
    
    rides_dict = {}
    rides = {}
    
    for index1 in range(length):
        index2 = index1 + 1
        Trip1 = dataset.iloc[index1]
        
        trips_processed += 1
        count = 0
        while (index2 < length):
            count += 1
            Trip2 = dataset.iloc[index2]
            if (Trip1['Passenger_count'] + Trip2['Passenger_count'] <= 3):
                if (time_difference(Trip1['lpep_pickup_datetime'], Trip2['lpep_pickup_datetime']) > delay or time_difference(Trip1['Lpep_dropoff_datetime'], Trip2['Lpep_dropoff_datetime']) > delay):
#                     print("Break, ", count)
                    break
            
            mergeable, dist = mergeable_check(Trip1, Trip2, delay)
            if (mergeable):
                rides.update({index1: {dist}})
                mergeable_trips += 1
            index2 = index2 + 1
        print(index1)
#         print('')
#         if (index1 % 200 == 0):
#             print(index1)
    return rides

start = time.time()
merged_trips = shared_trips_eval(dataset[:100000], 20.0, len(dataset[:100000]))
end = time.time()
# end = time.time()
# print('')
# print('Trips processed: ', trips_processed, 'Time Taken: ' ,end-start)

In [ ]:
print(merged_trips)

In [ ]:
max_matched = HopcroftKarp(merged_trips).maximum_matching(keys_only=True)

In [ ]:
print(sum(max_matched.values()))

In [ ]:
print('')
print('Trips processed: ', trips_processed) 
print('MergeAble Trips: ', len(max_matched)) 
print('Time Taken to process: ' ,end-start)
print('Total Distance Travelled:', dataset[:100000]['Trip_distance'].sum())
print('Total Miles saved: ', sum(max_matched.values()))
#print('Last Trip Processed: ')
#print(dataset.iloc[trips_processed])